In [1]:
import pickle
import pandas as pd


In [3]:
with open("model.bin", "rb") as f_in:
    dv, lr = pickle.load(f_in)

/Users/zar/.pyenv/versions/3.9.9/envs/mlops-env/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zar/.pyenv/versions/3.9.9/envs/mlops-env/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [10]:
year =  "2021"
month = "02"
df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year}-{month}.parquet')

dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

### Q1: What's the mean predicted duration for this dataset?

In [12]:
f"Mean predicted ride duration: {y_pred.mean()}"

'Mean predicted ride duration: 16.191691679979066'

In [16]:
df["ride_id"] = f"{year}/{month}_" + df.index.astype(str)
df["predicted_duration"] = y_pred

df_result = df.loc[:, ["ride_id", "predicted_duration"]]

output_file = f"fhv_{year}_{month}_prediction_results.parquet"
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [23]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 990113 entries, 1 to 1037691
Data columns (total 2 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             990113 non-null  object 
 1   predicted_duration  990113 non-null  float64
dtypes: float64(1), object(1)
memory usage: 22.7+ MB
